In [1]:
'''INITIAL CURVE SETUP
======================
1) DATA
========
- Historical Swap Data (swap_data)
- Normal Volatility Data (vol_data)
- Historical Zero Coupon Curves (zeroCoupons)
- Historical Forward Curves (forwardCurves)

2) FUNCTIONS
=============
- Zero Coupon interpolator (zeroCouponInterpolator)
- Forward Interpolator (forwardRate)
- Forward Swap Rate Interpolator (forwardSwapRate)

3) PACKAGES
=============
Set1: math, numpy, pandas, itertools, matplotlib
Set2: scipy.stats, scipy.optimize, datetime, time, relativedelta
Set3: nelson_siegel_svensson
Set4: prettytable
'''
# Initialize Curve Setup
%run CurveSetup.ipynb

In [2]:
# Extra Packages
import re
import scipy.optimize as opt
from lmfit import minimize, Parameters, Parameter, report_fit
from itertools import compress
import seaborn as sns
from sklearn.decomposition import PCA

In [3]:
'''FORWARD DATA HANDLING
=================='''
months =(forwardCurves.index.map(lambda x: (x.month)) == 12)
days = [day  in [29, 30, 31]for day in forwardCurves.index.day ]

# Filter the last day of each month
rawPCAData=forwardCurves.groupby([forwardCurves.index.year,
                                    forwardCurves.index.month]).last()
# Select the last day of the year
rawPCAData = rawPCAData[rawPCAData.index.map(lambda x: x[1] == 12)]

# Convert forwards to ratio
diffData = rawPCAData.pct_change()+1

# Remove NaN values
diffData = diffData.dropna()

'''PERFORM PCA ON LOG FORWARD RATIOS
======================================='''
pcaFit = PCA(n_components= 2)
components = pcaFit.fit_transform(diffData.transpose())

In [4]:
'''PRICERS CALIBRATION
========================='''
# Select Curve
zeroCouponCurve = zeroCoupons.loc['2019-12-31']
forwardCurve = forwardCurves.loc['2019-12-31']

def forwardSwapRateCalib(expiry, tenor):
    num = zeroCouponCurve[expiry] - zeroCouponCurve[expiry+tenor]
    
    PVBP = np.sum(zeroCouponCurve[expiry:expiry+tenor])
    
    return(num/PVBP)

' Black Pricer - Payer'
def blackPayer(N, S0, K, expiry, tenor, sigma):     
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:expiry+tenor])
    
    # Define Black Parameters
    T = expiry
    d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    price = N*PVBP * (S0*stats.norm.cdf(d1) - K*stats.norm.cdf(d2))
    return(price)


' Chi Square Pricer - Payer'
def chiSquarePayer(N, S0, K, expiry, tenor, sigma, beta, delta):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:expiry+tenor])
    
    # Parameters for input into Chi Square CDF
    T = expiry 
    d = (pow(K+delta,2 - 2*beta))/(pow(1 - beta,2)*pow(sigma,2)* T)
    b = 1/(1 - beta)
    f = (pow(S0+delta, 2 - 2*beta))/(pow(1 - beta, 2)*pow(sigma, 2)* T)

    # Calculate Price
    price = N * PVBP * ((S0+delta)*(1 - stats.ncx2.cdf(d, b+2, f)) - (K+delta)*(stats.ncx2.cdf(f, b, d)))
    return(price)

' Normal Pricer - Payer'
def normalPayer(N, S0, K, expiry, tenor, sigma):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:expiry+tenor])
    
     # Define Black Parameters
    T = expiry
    d1 = (S0 - K)/(sigma * sqrt(T))
    
    #Calculate Price
    price = N * PVBP * ((S0 - K)*stats.norm.cdf(d1) + (sigma * sqrt(T)* exp(-0.5 * pow(d1, 2)))/sqrt(2 * pi))
    return(price)

In [5]:
'''VECTORIZING ALL FUNCTIONS
======================================'''
zeroCouponVect = np.vectorize(zeroCouponInterpolator)
forwardRateVect = np.vectorize(forwardRate)
BlackVect = np.vectorize(blackPayer)
ChiSquareVect = np.vectorize(chiSquarePayer)
forwardSwapRateVect = np.vectorize(forwardSwapRateCalib)
normalPayerVect = np.vectorize(normalPayer)

In [6]:
'''SELECT SWAPTIONS
====================================='''
testVolData = vol_data.loc['2019-12-31']

'''FIND STRIKES FOR EACH SWAPTION'''
# Collect only annual Data
annualTenorNames = []
annualTenors = []

for string in testVolData.index:
    if 'M' not in string:
        annualTenorNames.append(string)
        annualTenors.append(testVolData.loc[string])

tenorMaturities =  [(int(re.findall('[0-9]+',string)[0]),int(re.findall('[0-9]+',string)[1]) ) 
                    for string in annualTenorNames]

expiries = [int(i[0]) for i in tenorMaturities]

tenors = [int(i[1]) for i in tenorMaturities]

#Obtain all strikes
strikes = forwardSwapRateVect(expiries, tenors) 

# Calculate value of all swaptions
normalPrices = normalPayerVect(1, strikes, strikes, expiries, tenors,annualTenors)

In [7]:
'''SIGMA ALPHA BETA FUNCTION
=============================
1) PCA parametrization'''
def sigmaAlphaBetaPCA(expiry, tenor, fZero, gamma, a, b, c, d, eta, betas, delta):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:tenor+expiry])

    # Calculate the forward swap rate
    swapForward = forwardSwapRateCalib(expiry, tenor)

    # Calculate the ZC bond value at T_alpha and T_beta
    zcAlpha = zeroCouponCurve[expiry]
    zcBeta = zeroCouponCurve[expiry+tenor]

    # Calculate weights
    '''To be confirmed
    ==================='''
    weights = [(swapForward*np.power(forwardCurve[i+1]+ delta, eta)/
                     np.power(swapForward+delta, eta)*(1 + forwardCurve[i+1]))* 
                    ((zcBeta/(zcAlpha-zcBeta)) + np.sum(zeroCouponCurve[i:expiry+tenor])/PVBP)
                    for i in list(range(expiry, expiry+tenor))]

    '''VOLATILITY FUNCTIONS
    ========================'''
    # f and g vectors 
    fValues = fZero + (1 - fZero)*np.exp(-gamma * np.arange(tenor + expiry + 1))
    gValues = ((a + b * np.arange(1, tenor + expiry + 1))* 
                np.exp(-c * np.arange(1, tenor + expiry + 1))) + d
    
    # Product of g and betas (Since they have the same index)
    gBeta1 = gValues * betas[:tenor+expiry,0]
    gBeta2 = gValues * betas[:tenor+expiry,1]
    
    '''INTEGRAL
    ============'''
    sigmaIntegral1 = np.sum([np.linalg.norm(fValues[i]*gBeta1[expiry-i:expiry+tenor-i]*weights)
                                for i in range(1, expiry+1)])
    sigmaIntegral2 = np.sum([np.linalg.norm(fValues[i]*gBeta1[expiry-i:expiry+tenor-i]*weights)
                                for i in range(1, expiry+1)])
  
    return(sigmaIntegral1 + sigmaIntegral2)

In [8]:
'''SIGMA ALPHA BETA FUNCTION
=============================
2) sigma, alpha, rho parametrization'''
def sigmaAlphaBeta(expiry, tenor, sigma1, sigma2, alpha1, alpha2, rho, eta, delta):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:tenor+expiry])

    # Calculate the forward swap rate
    swapForward = forwardSwapRateCalib(expiry, tenor)

    # Calculate the ZC bond value at T_alpha and T_beta
    zcAlpha = zeroCouponCurve[expiry]
    zcBeta = zeroCouponCurve[expiry+tenor]

    # Calculate delta weights
    '''To be confirmed'''
    deltaWeights = [(swapForward/(1 + forwardCurve[i+1]))* 
                    ((zcBeta/(zcAlpha-zcBeta)) + np.sum(zeroCouponCurve[i:expiry+tenor])/PVBP)
                    for i in list(range(expiry, expiry+tenor))]

    # Calculate Alpha Exponential 
    alpha1Exponential = np.exp(-alpha1 * np.arange(1, tenor + expiry + 1))
    alpha2Exponential = np.exp(-alpha2 * np.arange(1, tenor + expiry + 1))

    # Upsilon Terms (From volatility parametrization)
    upsilon1Terms = sqrt(1 - pow(rho, 2)) * sigma2 * alpha1Exponential 
    upsilon2Terms = sigma2 * alpha2Exponential + rho * sigma1 * alpha1Exponential

    # Combine vectors
    deltaWeightsCombination = list(itertools.combinations_with_replacement(deltaWeights, 2))
    forwardsCombination = list(itertools.combinations_with_replacement(
        np.power(np.array(forwardCurve[expiry:expiry+tenor])+delta, eta), 2))

    # Find the product of vectors
    deltaWeightsProduct = [np.product(elem) for elem in deltaWeightsCombination]
    forwardsProduct = [np.product(elem) for elem in forwardsCombination]

    # Define indices
    indices = list(itertools.combinations_with_replacement(np.arange(expiry, 
                                                        tenor + expiry), 2))

    #Integral of terms
    sigma1Integrals = [np.dot(upsilon1Terms[max(i, j) - min(i, j):max(i, j)], upsilon1Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon1Terms[:i], upsilon1Terms[:i]) for (i,j) in indices ]

    sigma2Integrals = [np.dot(upsilon2Terms[max(i, j) - min(i, j):max(i, j)], upsilon2Terms[:min(i, j)])
     if i != j 
     else np.dot(upsilon2Terms[:i], upsilon2Terms[:i]) for (i,j) in indices ]

    # Calculate different weights
    s1 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma1Integrals))
    s2 = sum(np.multiply(np.multiply(deltaWeightsProduct, forwardsProduct), sigma2Integrals))
    
    return((s2 + s1)/ pow(swapForward + delta, 2*eta))

In [9]:
''' Chi Square Pricer - Payer
=============================='''
def calibrationChiSquarePayer(N, S0, K, expiry, tenor, sigma1, sigma2, alpha1, alpha2, rho, eta, delta):
    # Find Zero Coupon Price at each date and calculate PVBP
    PVBP = np.sum(zeroCouponCurve[expiry:expiry+tenor])
    
    # Parameters for input into Chi Square CDF
    sigmaSquaredT =  sigmaAlphaBeta(expiry, tenor, sigma1, sigma2, alpha1, alpha2, rho, eta, delta)
    T = expiry
    d = (pow(K+delta,2 - 2*eta))/(pow(1 - eta,2)* sigmaSquaredT)
    b = 1/(1 - eta)
    f = (pow(S0+delta, 2 - 2*eta))/(pow(1 - eta, 2)* sigmaSquaredT)

    # Calculate Price
    price = N * PVBP * ((S0+delta)*(1 - stats.ncx2.cdf(d, b+2, f)) - (K+delta)*(stats.ncx2.cdf(f, b, d)))
    return(price)

# Vectorize Function
calibrationChiSquarePayerVect = np.vectorize(calibrationChiSquarePayer)

In [10]:
'''Define parameters for lmfit algorithm
========================================='''
# Set parameters and bounds
params = Parameters()
params.add('sigma1', value = 0.1, min = 0)
params.add('sigma2',value = 0.1, min = 0)
params.add('alpha1',value = 0.1, min =0)
params.add('alpha2',value = 0.1, min = 0)
params.add('rho', value = 0.5, min = -1, max = 1)
# params.add('eta', value = 0.5, min = 0, max = 1)

In [ ]:
'''CALIBRATE INDIVIDUAL PARAMETERS FROM PRICES
==============================================='''
# Select swaptions between 5X5 and 10X10
weights = np.array([((tenorMaturity[0] >= 5) & (tenorMaturity[0] <= 10 ) & 
                        (tenorMaturity[1] >= 5) & (tenorMaturity[1] <= 10 )) 
                    for tenorMaturity in tenorMaturities])

strikesCalib = np.array(list(compress(strikes, weights)))
expiriesCalib = list(compress(expiries, weights))
tenorCalib = list(compress(tenors, weights))
normalPricesCalib = list(compress(normalPrices, weights))

objectiveFunction = lambda test:  (calibrationChiSquarePayerVect(1, strikesCalib, 
                    strikesCalib, 
                    expiriesCalib, 
                    tenorCalib, 
                    test['sigma1'].value, 
                    test['sigma2'].value, 
                    test['alpha1'].value, 
                    test['alpha2'].value,
                    test['rho'].value,
                    0.7, 0.1) - normalPricesCalib)

startTime = time()
result = minimize(objectiveFunction, 
                  params, 
                  method = 'leastsq',
                  maxfev = 100000,
                  ftol = 1e-19)
endTime = time()

print('Optimization Complete in '+str(round((endTime - startTime)/60, 2)) + ' minutes.')

'''Print Calibration Results'''
headers = ["SS Error", "Sigma1", "Sigma2", "Alpha1", "Alpha2", "Rho", "Eta"]
t2 = PrettyTable()
t2.add_column(headers[0], [int(result.chisqr)])
t2.add_column(headers[1], [round(result.params['sigma1'].value, 5)])
t2.add_column(headers[2], [round(result.params['sigma2'].value, 5)])
t2.add_column(headers[3], [round(result.params['alpha1'].value, 5)])
t2.add_column(headers[4], [round(result.params['alpha2'].value, 5)])
t2.add_column(headers[5], [round(result.params['rho'].value, 5)])
# t2.add_column(headers[6], [round(result.params['eta'].value, 5)])
print(t2)

In [ ]:
'''HEATMAP OF ERRORS
====================='''
weights = np.array([((tenorMaturity[0] >= 5) & (tenorMaturity[0] <= 10 ) & 
                        (tenorMaturity[1] >= 5) & (tenorMaturity[1] <= 10 )) 
                    for tenorMaturity in tenorMaturities])

strikesCalib = np.array(list(compress(strikes, weights)))
expiriesCalib = list(compress(expiries, weights))
tenorsCalib = list(compress(tenors, weights))
normalPricesCalib = list(compress(normalPrices, weights))

percentageErrors = np.absolute((calibrationChiSquarePayerVect(1, strikesCalib, 
                    strikesCalib, 
                    expiriesCalib, 
                    tenorsCalib, 
                    result.params['sigma1'].value, 
                    result.params['sigma2'].value, 
                    result.params['alpha1'].value, 
                    result.params['alpha2'].value,
                    result.params['rho'].value,
                    0.7,
                    0.1)*100/normalPricesCalib)-100)

# Construct Data
heatmapData = pd.DataFrame([expiriesCalib, tenorsCalib, percentageErrors]).transpose()
heatmapData.columns =  ['Expiry', 'Tenor', 'Error']
heatmapData = heatmapData.pivot(index = 'Expiry', columns = 'Tenor', values = 'Error')

# Constructing the plot
fig, ax = plt.subplots(figsize = (40, 36), facecolor='#444444')

# Add Title
title = "Percentage Calibration Errors"

# Set the title and distance from the plo
ttl = ax.title
ttl.set_position([0.5, 1.05])
ax.invert_yaxis()
ax.set_title(title, color='black', fontsize = 50)

# Define labels
labels = np.array(percentageErrors.transpose()).reshape(heatmapData.shape)

# Use seaborn 
plot = sns.heatmap(heatmapData,
            cmap = "viridis",
            cbar= False,
            annot = True,
           linewidths = 0.3, 
           ax = ax,
           annot_kws={'size':40})

plot.axes.xaxis.set_ticks_position('top')
plt.xlabel('Tenor', fontsize = 40, color = 'black')
plt.ylabel('Expiry', fontsize = 40, color = 'black')

ax.tick_params(labelcolor='black',
              labelsize = 38)
plt.savefig("Calibration Errors")